 Formula 1 Winner Prediction - Step 2: Data Preprocessing

In [1]:
import pandas as pd
import numpy as np

In [9]:
drivers = pd.read_csv('../data/drivers.csv')
constructors = pd.read_csv('../data/constructors.csv')
circuits = pd.read_csv('../data/circuits.csv')
races = pd.read_csv('../data/races.csv')
results = pd.read_csv('../data/results.csv')
qualifying = pd.read_csv('../data/qualifying.csv', low_memory=False)


In [10]:
races['date'] = pd.to_datetime(races['date'])
drivers['driver_name'] = drivers['forename'] + ' ' + drivers['surname']

In [11]:
df = results.merge(drivers, on='driverId', how='left')
df = df.merge(constructors, on='constructorId', how='left')
df = df.merge(races[['raceId', 'year', 'round', 'circuitId', 'date']], on='raceId', how='left')
df = df.merge(circuits[['circuitId', 'name', 'location', 'country']], on='circuitId', how='left')

In [12]:
df['is_winner'] = np.where(df['positionOrder'] == 1, 1, 0)
qual_df = qualifying[['raceId', 'driverId', 'position']].rename(columns={'position': 'qualifying_position'})
df = df.merge(qual_df, on=['raceId', 'driverId'], how='left')
df['qualifying_position'] = df['qualifying_position'].fillna(20)
df = df.sort_values('date')

In [13]:
df = df.rename(columns={'name_x': 'race_name', 'name_y': 'circuit_name', 'constructorRef': 'constructor_name'})
df['circuit_driver_key'] = df['driverId'].astype(str) + '_' + df['circuitId'].astype(str)


In [14]:
df['past_wins_at_circuit'] = 0
circuit_driver_wins = {}
for idx, row in df.iterrows():
    key = row['circuit_driver_key']
    if key not in circuit_driver_wins:
        circuit_driver_wins[key] = 0
    df.at[idx, 'past_wins_at_circuit'] = circuit_driver_wins[key]
    if row['is_winner'] == 1:
        circuit_driver_wins[key] += 1


In [15]:
# Add a unique key for each driver+circuit combination
df['circuit_driver_key'] = df['driverId'].astype(str) + '_' + df['circuitId'].astype(str)

# Create the past_wins_at_circuit column
df['past_wins_at_circuit'] = 0
circuit_driver_wins = {}

for idx, row in df.iterrows():
    key = row['circuit_driver_key']
    if key not in circuit_driver_wins:
        circuit_driver_wins[key] = 0
    df.at[idx, 'past_wins_at_circuit'] = circuit_driver_wins[key]
    if row['is_winner'] == 1:
        circuit_driver_wins[key] += 1


In [16]:
final_df = df[[
    'raceId', 'year', 'round', 'date',
    'driverId', 'driver_name', 'constructorId', 'constructor_name',
    'positionOrder', 'grid', 'qualifying_position', 'is_winner',
    'circuitId', 'circuit_name', 'country', 'past_wins_at_circuit'
]]
final_df.to_csv("processed_f1_data.csv", index=False)
print(" Data Preprocessing Complete.")

 Data Preprocessing Complete.
